In [1]:
from os.path import join
import numpy as np
import pandas as pd

from feature.dataset import make_dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from category_encoders import *

train_path = join('jeju_data', 'train_new.parquet')
test_path = join('jeju_data', 'test_new.parquet')
holiday_path = join('jeju_data', '국가공휴일.csv')

sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')

x_train, y_train, test = make_dataset(train_path, test_path, holiday_path)

Start time:  2022-11-09 15:46:28.100229
Train dataset success !
Test dataset success !
End time: 2022-11-09 15:48:48.760135
Play time:  0:02:20.659906


In [28]:
train = pd.read_parquet('train_48_T40.parquet')
test = pd.read_parquet('test_48_T40.parquet')

In [29]:
X = train.drop(['target'], axis=1)
y = train['target']

In [46]:
X.columns

Index(['day_of_week', 'base_hour', 'lane_count', 'road_rating', 'road_name',
       'multi_linked', 'connect_code', 'maximum_speed_limit',
       'weight_restricted', 'road_type', 'start_node_name', 'start_latitude',
       'start_longitude', 'start_turn_restricted', 'end_node_name',
       'end_latitude', 'end_longitude', 'end_turn_restricted', 'sin_time',
       'cos_time', 'group_time', 'month', 'week', 'post_holiday',
       'pre_holiday', 'holiday', 'season', 'vacation', 'distance', 'jeju_dist',
       'seogwi_dist', 'hanra_dist', 'sungsan_dist', 'joongmoon_dist',
       'node_TF', 'diff', 'road_name_set', 'section_speed_time',
       'start_speed_time', 'end_speed_time', 'section_speed', 'start_speed',
       'end_speed', 'end_cartesian', 'tour_count', 'turn_restricted', 'rest',
       'location_cluster'],
      dtype='object')

In [47]:
X.head()

,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,weight_restricted,road_type,...,start_speed_time,end_speed_time,section_speed,start_speed,end_speed,end_cartesian,tour_count,turn_restricted,rest,location_cluster
0,1,17,1,106,6,0,0,60.0,32400.0,3,...,46.450450,48.659670,49.105982,48.697943,50.298219,324,9,0,0.0,2
1,1,21,2,103,30,0,0,60.0,0.0,0,...,26.562992,26.562992,47.203323,26.400712,26.400712,532,15,1,0.0,3
2,4,7,2,103,32,0,0,80.0,0.0,0,...,60.135135,66.588964,56.858438,59.101720,65.118140,146,5,0,1.0,0
3,0,13,2,107,52,0,0,50.0,0.0,0,...,20.789883,23.299611,25.030004,23.755158,25.445418,7,15,0,0.0,1
4,6,8,2,103,31,0,0,80.0,0.0,0,...,40.518182,49.972727,51.188650,39.873670,39.873670,394,12,0,1.0,0


In [50]:
df = pd.concat([X, y], axis=1)
df.corr()
# df.corr().to_csv('public3.07 correlation.csv')

,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,weight_restricted,road_type,...,end_speed_time,section_speed,start_speed,end_speed,end_cartesian,tour_count,turn_restricted,rest,location_cluster,target
day_of_week,1.000000,0.004889,0.001426,-0.002274,0.000965,-0.000084,-0.000002,0.001651,0.000158,0.000429,...,-0.000003,0.001805,0.001776,0.001720,0.001332,0.000008,0.001090,0.427658,0.001977,0.006397
base_hour,0.004889,1.000000,-0.029194,0.031658,-0.020040,0.005711,-0.002649,-0.036756,-0.003231,-0.007880,...,-0.175974,-0.025953,-0.025166,-0.024868,-0.023517,0.005212,-0.014949,0.020477,-0.031788,-0.159407
lane_count,0.001426,-0.029194,1.000000,-0.095717,0.011352,-0.026555,-0.029290,0.384002,-0.177224,-0.050715,...,-0.172446,-0.073179,-0.163500,-0.164086,0.209519,0.113366,0.361270,-0.000909,0.248377,-0.144256
road_rating,-0.002274,0.031658,-0.095717,1.000000,-0.199358,0.024218,-0.054160,-0.327474,-0.118630,-0.125618,...,-0.275517,-0.345963,-0.299348,-0.298064,-0.236377,0.179038,-0.034892,-0.005431,-0.138706,-0.261693
road_name,0.000965,-0.020040,0.011352,-0.199358,1.000000,-0.022880,0.026512,0.096440,-0.422081,-0.532391,...,-0.019047,-0.019101,-0.016802,-0.018331,0.039509,0.151932,0.083281,0.000029,0.065659,-0.013875
multi_linked,-0.000084,0.005711,-0.026555,0.024218,-0.022880,1.000000,-0.001111,-0.020245,-0.008790,0.042977,...,-0.009282,-0.003808,-0.009614,-0.009562,-0.011555,-0.018864,-0.008592,0.000034,0.005702,-0.008408
connect_code,-0.000002,-0.002649,-0.029290,-0.054160,0.026512,-0.001111,1.000000,-0.015190,-0.020491,-0.025846,...,0.057797,0.065331,0.054734,0.067010,0.034831,-0.018763,-0.020029,0.001203,0.042713,0.048348
maximum_speed_limit,0.001651,-0.036756,0.384002,-0.327474,0.096440,-0.020245,-0.015190,1.000000,0.085080,0.059511,...,0.437806,0.586062,0.486769,0.484095,0.263963,-0.204260,0.118610,0.006086,0.205875,0.425715
weight_restricted,0.000158,-0.003231,-0.177224,-0.118630,-0.422081,-0.008790,-0.020491,0.085080,1.000000,0.792803,...,0.325522,0.293173,0.326668,0.334956,-0.114564,-0.174395,-0.142706,0.002555,-0.081639,0.294092
road_type,0.000429,-0.007880,-0.050715,-0.125618,-0.532391,0.042977,-0.025846,0.059511,0.792803,1.000000,...,0.219752,0.276487,0.222298,0.229993,-0.018000,-0.187102,-0.123900,0.000949,0.023749,0.200840


In [33]:
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

params = {'n_estimators': 4693, 'max_depth': 15, 'min_child_weight': 4, 'gamma': 1, 'learning_rate': 0.018, 'colsample_bytree': 0.9015018647603987, 'lambda': 1.386941960727803, 'alpha': 0.10534800837686535, 'subsample': 1.0}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

folds = []

for train_idx, val_idx in skf.split(X, y):
    folds.append((train_idx, val_idx))

XGB_model= {}

for f in range(5):
      print(f'===================================={f+1}============================================')
      train_idx, val_idx = folds[f]
      
      x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[val_idx], y.iloc[train_idx], y.iloc[val_idx]
      
      XGB = XGBRegressor(**params, tree_method='gpu_hist', gpu_id=0)
      XGB.fit(x_train, y_train)
      
      y_pred = XGB.predict(x_val)
      mae = mean_absolute_error(y_val, y_pred)
      print(f"{f + 1} Fold MAE = {mae}")
      XGB_model[f] = XGB
      print(f'================================================================================\n\n')
              


sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')

for fold in range(5):
    sample_submission['target'] += XGB_model[fold].predict(test)/5    
    

c:\mb_job\project\Competition\DACON\jeju_traffic_prediction\jeju_project\lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


====================================1============================================
1 Fold MAE = 2.953515075024202


====================================2============================================
2 Fold MAE = 2.952919077738026


====================================3============================================
3 Fold MAE = 2.9527815800524926


====================================4============================================
4 Fold MAE = 2.95185732043858


====================================5============================================
5 Fold MAE = 2.9510384626180612




In [35]:
# sample_submission.to_csv("./xgb_fold_mb.csv", index=False)
round_target = round(sample_submission['target'])
sample_submission['target'] = round_target
sample_submission.to_csv("./xgb_fold_mb_round.csv", index=False)

In [36]:
df_imp = pd.DataFrame({'imp':XGB.feature_importances_}, index = XGB.feature_names_in_)
df_imp = df_imp[df_imp.imp > 0].sort_values('imp').copy()
df_imp

,imp
day_of_week,0.000602
post_holiday,0.000628
sin_time,0.000814
rest,0.000820
holiday,0.000870
pre_holiday,0.000875
base_hour,0.000912
cos_time,0.000936
vacation,0.000940
month,0.001041
